In [1]:
# to disable GPU
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
import os
os.chdir('T:/Dropbox/Projects/LSTM-forecast/src')
import featureEng as fe
import processStocks as ps
import lstm
import getStocks
import visualize
import predicts
import pandas as pd

Using TensorFlow backend.


In [3]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [4]:
# view tenslow enabled devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17800576281146000593
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6767002255
locality {
  bus_id: 1
}
incarnation: 17113697049448286916
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:65:00.0, compute capability: 6.1"
]


# User parameters

In [5]:
####TODO: reshaping so we can plot various n_in, n_out
# model seems to work, but cant redim for plot
ticker = 'unh'
n_in = 5
n_out = 1
target = 'd1close'
rngSpt = 20 # number of random stocks to select as the supporting data features

In [6]:
import imp
imp.reload(getStocks)

<module 'getStocks' from 'T:\\Dropbox\\Projects\\LSTM-forecast\\src\\getStocks.py'>

In [7]:
# collect a bunch of data from the market, save it to directory
# getStocks.get_mkt_data(source='yahoo')

# Load Data
### Several methods available here:
* Load a single stock
* get a refresh of the stock
* load a single stock as the target while using many other stocks as additional features

In [8]:
# load dataset
# dataset = getStocks.get_single(ticker=ticker, save=True)
# dataset = getStocks.load_single(ticker)
dataset = getStocks.join_tgt_spt(target_ticker='UNH', number_spt=rngSpt)
dataset.reset_index(level=0, inplace=True)
dataset.rename(columns={'Adj Close':'AdjCls'}, inplace=True)

In [9]:
dataset.head()

,Date,Open,High,Low,Close,AdjCls,Volume,ECLOpen,ECLHigh,ECLLow,ECLClose,ECLAdj Close,ECLVolume,NKEOpen,NKEHigh,NKELow,NKEClose,NKEAdj Close,NKEVolume,MACOpen,MACHigh,MACLow,MACClose,MACAdj Close,MACVolume,KSUOpen,KSUHigh,KSULow,KSUClose,KSUAdj Close,KSUVolume,DOpen,DHigh,DLow,DClose,DAdj Close,DVolume,KMXOpen,KMXHigh,KMXLow,KMXClose,KMXAdj Close,KMXVolume,ORLYOpen,ORLYHigh,ORLYLow,ORLYClose,ORLYAdj Close,ORLYVolume,LKQOpen,LKQHigh,LKQLow,LKQClose,LKQAdj Close,LKQVolume,PHOpen,PHHigh,PHLow,PHClose,PHAdj Close,PHVolume,CAHOpen,CAHHigh,CAHLow,CAHClose,CAHAdj Close,CAHVolume,QCOMOpen,QCOMHigh,QCOMLow,QCOMClose,QCOMAdj Close,QCOMVolume,COTYOpen,COTYHigh,COTYLow,COTYClose,COTYAdj Close,COTYVolume,DFSOpen,DFSHigh,DFSLow,DFSClose,DFSAdj Close,DFSVolume,AAPLOpen,AAPLHigh,AAPLLow,AAPLClose,AAPLAdj Close,AAPLVolume,LUVOpen,LUVHigh,LUVLow,LUVClose,LUVAdj Close,LUVVolume,TAPOpen,TAPHigh,TAPLow,TAPClose,TAPAdj Close,TAPVolume,FITBOpen,FITBHigh,FITBLow,FITBClose,FITBAdj Close,FITBVolume,DVAOpen,DVAHigh,DVALow,DVAClose,DVAAdj Close,DVAVolume,GOOGOpen,GOOGHigh,GOOGLow,GOOGClose,GOOGAdj Close,GOOGVolume,MMCOpen,MMCHigh,MMCLow,MMCClose,MMCAdj Close,MMCVolume
0,1984-10-17,0.144531,0.148438,0.144531,0.144531,0.126899,9868800,1.562500,1.578125,1.554688,1.578125,0.971575,2828800,0.128906,0.130859,0.128906,0.130859,0.098292,966400,NaN,NaN,NaN,NaN,NaN,NaN,6.916667,6.958333,6.875000,6.937500,2.771326,181200,8.916667,9.125000,8.875000,9.041667,1.537180,435000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.604800,1.604800,1.579725,1.604800,0.888511,63900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.444196,0.446429,0.444196,0.444196,0.363467,39160800.0,0.779150,0.823045,0.779150,0.779150,0.693485,6972300,7.4375,7.4375,7.4375,7.4375,3.989465,12600,1.307118,1.307118,1.307118,1.307118,0.628810,21100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.291667,4.312500,4.270833,4.270833,1.633721,1231200.0
1,1984-10-18,0.148438,0.156250,0.148438,0.148438,0.130329,5324800,1.578125,1.593750,1.570313,1.578125,0.971575,1112000,0.130859,0.130859,0.128906,0.128906,0.096825,2579200,NaN,NaN,NaN,NaN,NaN,NaN,6.937500,6.979167,6.916667,6.979167,2.787970,21000,9.041667,9.250000,9.041667,9.208333,1.565515,545600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.604800,1.604800,1.504500,1.529575,0.846862,132600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.457589,0.459821,0.457589,0.457589,0.374425,61790400.0,0.839506,0.894376,0.839506,0.883402,0.786275,4692700,7.6875,7.6875,7.6875,7.6875,4.123566,50400,1.313621,1.333130,1.313621,1.333130,0.641324,147700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.270833,4.385417,4.260417,4.385417,1.677553,404400.0
2,1984-10-19,0.148438,0.148438,0.144531,0.144531,0.126899,3043200,1.578125,1.609375,1.578125,1.593750,0.981195,164800,0.128906,0.132813,0.126953,0.126953,0.095357,6419200,NaN,NaN,NaN,NaN,NaN,NaN,7.000000,7.125000,7.000000,7.041667,2.812938,272400,9.250000,9.416667,8.375000,9.250000,1.572599,639600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.529575,1.529575,1.404200,1.429275,0.791330,65300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.457589,0.488839,0.455357,0.457589,0.374425,81530400.0,0.883402,0.905350,0.850480,0.861454,0.766740,5207500,7.6875,7.7500,7.5625,7.5625,4.056516,56000,1.333130,1.352639,1.326627,1.339633,0.644452,639900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.416667,4.541667,4.416667,4.541667,1.737323,1123200.0
3,1984-10-22,0.144531,0.148438,0.144531,0.144531,0.126899,2326400,1.593750,1.609375,1.593750,1.609375,0.990815,3011200,0.126953,0.128906,0.125000,0.126953,0.095357,3980800,NaN,NaN,NaN,NaN,NaN,NaN,7.041667,7.083333,6.916667,6.916667,2.763003,40200,9.166667,9.250000,9.125000,9.125000,1.551347,336200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [10]:
for col in dataset.columns:
    print(col)

Date
Open
High
Low
Close
AdjCls
Volume
ECLOpen
ECLHigh
ECLLow
ECLClose
ECLAdj Close
ECLVolume
NKEOpen
NKEHigh
NKELow
NKEClose
NKEAdj Close
NKEVolume
MACOpen
MACHigh
MACLow
MACClose
MACAdj Close
MACVolume
KSUOpen
KSUHigh
KSULow
KSUClose
KSUAdj Close
KSUVolume
DOpen
DHigh
DLow
DClose
DAdj Close
DVolume
KMXOpen
KMXHigh
KMXLow
KMXClose
KMXAdj Close
KMXVolume
ORLYOpen
ORLYHigh
ORLYLow
ORLYClose
ORLYAdj Close
ORLYVolume
LKQOpen
LKQHigh
LKQLow
LKQClose
LKQAdj Close
LKQVolume
PHOpen
PHHigh
PHLow
PHClose
PHAdj Close
PHVolume
CAHOpen
CAHHigh
CAHLow
CAHClose
CAHAdj Close
CAHVolume
QCOMOpen
QCOMHigh
QCOMLow
QCOMClose
QCOMAdj Close
QCOMVolume
COTYOpen
COTYHigh
COTYLow
COTYClose
COTYAdj Close
COTYVolume
DFSOpen
DFSHigh
DFSLow
DFSClose
DFSAdj Close
DFSVolume
AAPLOpen
AAPLHigh
AAPLLow
AAPLClose
AAPLAdj Close
AAPLVolume
LUVOpen
LUVHigh
LUVLow
LUVClose
LUVAdj Close
LUVVolume
TAPOpen
TAPHigh
TAPLow
TAPClose
TAPAdj Close
TAPVolume
FITBOpen
FITBHigh
FITBLow
FITBClose
FITBAdj Close
FITBVolume
DVAOpen
DVAHig

# Feature Engineering
* Add day of week feature
* Move target feature to end of dataframe

In [11]:
## Generate new features
dataset = fe.derivative(dataset, drop_na = True)
dataset = fe.weekDay(dataset)
features = list(dataset.columns)
# move target to position [-1]
features.remove(target)
features.append(target)
features=list(features)
dataset=dataset[features]

In [12]:
# Plot the features
# visualize.plot_features(dataset)

In [13]:
dataset.head(2)

,Open,High,Low,Close,AdjCls,Volume,ECLOpen,ECLHigh,ECLLow,ECLClose,ECLAdj Close,ECLVolume,NKEOpen,NKEHigh,NKELow,NKEClose,NKEAdj Close,NKEVolume,MACOpen,MACHigh,MACLow,MACClose,MACAdj Close,MACVolume,KSUOpen,KSUHigh,KSULow,KSUClose,KSUAdj Close,KSUVolume,DOpen,DHigh,DLow,DClose,DAdj Close,DVolume,KMXOpen,KMXHigh,KMXLow,KMXClose,KMXAdj Close,KMXVolume,ORLYOpen,ORLYHigh,ORLYLow,ORLYClose,ORLYAdj Close,ORLYVolume,LKQOpen,LKQHigh,LKQLow,LKQClose,LKQAdj Close,LKQVolume,PHOpen,PHHigh,PHLow,PHClose,PHAdj Close,PHVolume,CAHOpen,CAHHigh,CAHLow,CAHClose,CAHAdj Close,CAHVolume,QCOMOpen,QCOMHigh,QCOMLow,QCOMClose,QCOMAdj Close,QCOMVolume,COTYOpen,COTYHigh,COTYLow,COTYClose,COTYAdj Close,COTYVolume,DFSOpen,DFSHigh,DFSLow,DFSClose,DFSAdj Close,DFSVolume,AAPLOpen,AAPLHigh,AAPLLow,AAPLClose,AAPLAdj Close,AAPLVolume,LUVOpen,LUVHigh,LUVLow,LUVClose,LUVAdj Close,LUVVolume,TAPOpen,TAPHigh,TAPLow,TAPClose,TAPAdj Close,TAPVolume,FITBOpen,FITBHigh,FITBLow,FITBClose,FITBAdj Close,FITBVolume,DVAOpen,DVAHigh,DVALow,DVAClose,DVAAdj Close,DVAVolume,GOOGOpen,GOOGHigh,GOOGLow,GOOGClose,GOOGAdj Close,GOOGVolume,MMCOpen,MMCHigh,MMCLow,MMCClose,MMCAdj Close,MMCVolume,d2close,d1vol,d2vol,M,T,W,Th,F,d1close
0,63.700001,64.209999,63.150002,64.150002,59.675560,4379500,84.040001,84.879997,83.400002,84.760002,80.551239,1292800,30.650000,30.959999,30.514999,30.879999,29.084646,7832600,61.139999,62.930000,60.849998,62.799999,50.206482,1020300.0,107.529999,111.750000,106.940002,111.449997,105.607903,784600,55.299999,56.490002,55.220001,56.400002,47.802444,3287300,45.849998,47.090000,45.779999,46.990002,46.990002,993300.0,110.290001,111.809998,109.860001,111.449997,111.449997,470400.0,25.209999,25.389999,24.33,25.030001,25.030001,861000.0,96.919998,98.410004,96.290001,98.019997,89.894325,837400.0,47.130001,47.950001,46.849998,47.900002,43.630695,2224100,61.220001,62.119999,60.75,61.959999,54.088882,11578600.0,17.500000,17.520000,16.900000,17.360001,16.041420,46263100.0,47.689999,48.200001,47.250000,48.099998,44.176403,2267700.0,61.785713,62.448570,61.250000,62.279999,57.028923,71458100.0,13.85,14.06,13.81,13.98,13.480129,5776400,49.560001,49.939999,49.180000,49.84,45.498154,1466700,18.250000,18.49,18.150000,18.450001,16.477245,8225900,64.855003,65.074997,64.180000,64.930000,64.930000,1214400.0,431.682312,436.987823,429.953552,435.666412,435.666412,4166600.0,39.810001,40.380001,39.590000,40.310001,36.653500,1992600.0,0.419998,488400.0,2660700.0,0.0,0.0,0.0,1.0,0.0,0.320000
1,64.129997,64.449997,63.590000,63.799999,59.349964,3122600,84.370003,84.699997,83.959999,84.029999,80.074776,898600,30.815001,31.200001,30.805000,30.945000,29.145864,5222400,62.630001,63.290001,61.950001,62.250000,49.766769,1177400.0,111.120003,112.449997,110.230003,110.599998,104.802467,514600,56.369999,56.590000,55.950001,56.040001,47.497322,2647300,46.820000,47.310001,46.369999,46.410000,46.410000,1145100.0,111.180000,112.470001,110.970001,111.559998,111.559998,446100.0,24.969999,25.360001,24.91,25.299999,25.299999,1159200.0,96.480003,98.010002,96.480003,97.410004,89.334915,477700.0,47.810001,48.369999,47.689999,47.830002,43.566940,1866300,61.790001,61.930000,61.34,61.380001,53.582565,9871600.0,17.450001,17.549999,17.200001,17.520000,16.189266,9049500.0,48.020000,48.130001,46.970001,47.340000,43.478394,3007900.0,62.200001,62.327145,61.214287,61.435715,56.255829,67966500.0,13.92,14.01,13.83,13.87,13.374065,5130500,49.840000,50.110001,49.450001,49.59,45.269932,1143500,18.469999,18.49,18.200001,18.309999,16.352211,8866600,64.894997,65.214996,64.230003,64.415001,64.415001,742400.0,437.206390,439.491516,434.240692,434.692749,434.692749,4519100.0,40.240002,40.400002,39.880001,39.950001,36.326149,1413700.0,-0.670003,-1256900.0,-1745300.0,0.0,0.0,0.0,0.0,1.0,-0.350003


# Format as Supervised Learning Problem

In [14]:
# frame as supervised learning
# this will be for scaling the data to the window scaled_in to scaled_out
reframed=ps.series_to_supervised(dataset, n_in=n_in, n_out=n_out, 
                                   features=features)

starting Series to supverised conversion...


In [15]:
# drop all but the 'target' from the predictor set
# this might be able to take an array for multi-output
reframed=ps.frame_targets(reframed, features, n_out,target=target)

#scaler = MinMaxScaler(feature_range=(-1, 1))
#scaled = scaler.fit_transform(reframed)
# pass the feature in the sequence that we dont want scaled

# Scaling - several methods
* yet to be determined if scaling improves performance

In [16]:
imp.reload(ps)

<module 'processStocks' from 'T:\\Dropbox\\Projects\\LSTM-forecast\\src\\processStocks.py'>

In [17]:
#reframed=reframed.iloc[:10,:] 
#test/debug
#dataset=reframed

In [18]:
#scaled=ps.scale_sequence(reframed, 
#                         features, 
#                         scaleTarget=True, 
#                         target=target)
# or skip the scaling
scaled=reframed

In [19]:
#getStocks.saveScaled(scaled, n_in, n_out, ticker)
# load scaled 

#scaled=pd.read_csv('../data/astc/astc_29092017_scaled_30_30.csv')

# Cross Validation Setup

In [20]:
# put in order of time (t-n_in should be on left of df, with t+ on the right)

# split into train, validation, test
train, validation, test = lstm.tscv(scaled, train=0.7, validation=0.25)

In [21]:
# split into input and outputs
# the last n columns are the output variable

train_X, train_y = train[:, :-n_out], train[:, -n_out:]
X_validation, Y_validation = validation[:, :-n_out], validation[:, -n_out:]
test_X, test_y = test[:, :-n_out], test[:, -n_out:]

# Reshape to 3d tenser

In [22]:
scaled.head()

,Open(t-05),High(t-05),Low(t-05),Close(t-05),AdjCls(t-05),Volume(t-05),ECLOpen(t-05),ECLHigh(t-05),ECLLow(t-05),ECLClose(t-05),ECLAdj Close(t-05),ECLVolume(t-05),NKEOpen(t-05),NKEHigh(t-05),NKELow(t-05),NKEClose(t-05),NKEAdj Close(t-05),NKEVolume(t-05),MACOpen(t-05),MACHigh(t-05),MACLow(t-05),MACClose(t-05),MACAdj Close(t-05),MACVolume(t-05),KSUOpen(t-05),KSUHigh(t-05),KSULow(t-05),KSUClose(t-05),KSUAdj Close(t-05),KSUVolume(t-05),DOpen(t-05),DHigh(t-05),DLow(t-05),DClose(t-05),DAdj Close(t-05),DVolume(t-05),KMXOpen(t-05),KMXHigh(t-05),KMXLow(t-05),KMXClose(t-05),KMXAdj Close(t-05),KMXVolume(t-05),ORLYOpen(t-05),ORLYHigh(t-05),ORLYLow(t-05),ORLYClose(t-05),ORLYAdj Close(t-05),ORLYVolume(t-05),LKQOpen(t-05),LKQHigh(t-05),LKQLow(t-05),LKQClose(t-05),LKQAdj Close(t-05),LKQVolume(t-05),PHOpen(t-05),PHHigh(t-05),PHLow(t-05),PHClose(t-05),PHAdj Close(t-05),PHVolume(t-05),CAHOpen(t-05),CAHHigh(t-05),CAHLow(t-05),CAHClose(t-05),CAHAdj Close(t-05),CAHVolume(t-05),QCOMOpen(t-05),QCOMHigh(t-05),QCOMLow(t-05),QCOMClose(t-05),QCOMAdj Close(t-05),QCOMVolume(t-05),COTYOpen(t-05),COTYHigh(t-05),COTYLow(t-05),COTYClose(t-05),COTYAdj Close(t-05),COTYVolume(t-05),DFSOpen(t-05),DFSHigh(t-05),DFSLow(t-05),DFSClose(t-05),DFSAdj Close(t-05),DFSVolume(t-05),AAPLOpen(t-05),AAPLHigh(t-05),AAPLLow(t-05),AAPLClose(t-05),AAPLAdj Close(t-05),AAPLVolume(t-05),LUVOpen(t-05),LUVHigh(t-05),LUVLow(t-05),LUVClose(t-05),LUVAdj Close(t-05),LUVVolume(t-05),TAPOpen(t-05),TAPHigh(t-05),TAPLow(t-05),TAPClose(t-05),TAPAdj Close(t-05),TAPVolume(t-05),FITBOpen(t-05),FITBHigh(t-05),FITBLow(t-05),FITBClose(t-05),FITBAdj Close(t-05),FITBVolume(t-05),DVAOpen(t-05),DVAHigh(t-05),DVALow(t-05),DVAClose(t-05),DVAAdj Close(t-05),DVAVolume(t-05),GOOGOpen(t-05),GOOGHigh(t-05),GOOGLow(t-05),GOOGClose(t-05),GOOGAdj Close(t-05),GOOGVolume(t-05),MMCOpen(t-05),MMCHigh(t-05),MMCLow(t-05),MMCClose(t-05),MMCAdj Close(t-05),MMCVolume(t-05),d2close(t-05),d1vol(t-05),d2vol(t-05),M(t-05),T(t-05),W(t-05),Th(t-05),F(t-05),d1close(t-05),Open(t-04),High(t-04),Low(t-04),Close(t-04),AdjCls(t-04),Volume(t-04),ECLOpen(t-04),ECLHigh(t-04),ECLLow(t-04),ECLClose(t-04),ECLAdj Close(t-04),ECLVolume(t-04),NKEOpen(t-04),NKEHigh(t-04),NKELow(t-04),NKEClose(t-04),NKEAdj Close(t-04),NKEVolume(t-04),MACOpen(t-04),MACHigh(t-04),MACLow(t-04),MACClose(t-04),MACAdj Close(t-04),MACVolume(t-04),KSUOpen(t-04),KSUHigh(t-04),KSULow(t-04),KSUClose(t-04),KSUAdj Close(t-04),KSUVolume(t-04),DOpen(t-04),DHigh(t-04),DLow(t-04),DClose(t-04),DAdj Close(t-04),DVolume(t-04),KMXOpen(t-04),KMXHigh(t-04),KMXLow(t-04),KMXClose(t-04),KMXAdj Close(t-04),KMXVolume(t-04),ORLYOpen(t-04),ORLYHigh(t-04),ORLYLow(t-04),ORLYClose(t-04),ORLYAdj Close(t-04),ORLYVolume(t-04),LKQOpen(t-04),LKQHigh(t-04),LKQLow(t-04),LKQClose(t-04),LKQAdj Close(t-04),LKQVolume(t-04),PHOpen(t-04),PHHigh(t-04),PHLow(t-04),PHClose(t-04),PHAdj Close(t-04),PHVolume(t-04),CAHOpen(t-04),CAHHigh(t-04),CAHLow(t-04),CAHClose(t-04),CAHAdj Close(t-04),CAHVolume(t-04),QCOMOpen(t-04),QCOMHigh(t-04),QCOMLow(t-04),QCOMClose(t-04),QCOMAdj Close(t-04),QCOMVolume(t-04),COTYOpen(t-04),COTYHigh(t-04),COTYLow(t-04),COTYClose(t-04),COTYAdj Close(t-04),COTYVolume(t-04),DFSOpen(t-04),DFSHigh(t-04),DFSLow(t-04),DFSClose(t-04),DFSAdj Close(t-04),DFSVolume(t-04),AAPLOpen(t-04),AAPLHigh(t-04),AAPLLow(t-04),AAPLClose(t-04),AAPLAdj Close(t-04),AAPLVolume(t-04),LUVOpen(t-04),LUVHigh(t-04),LUVLow(t-04),LUVClose(t-04),LUVAdj Close(t-04),LUVVolume(t-04),TAPOpen(t-04),TAPHigh(t-04),TAPLow(t-04),TAPClose(t-04),TAPAdj Close(t-04),TAPVolume(t-04),FITBOpen(t-04),FITBHigh(t-04),FITBLow(t-04),FITBClose(t-04),FITBAdj Close(t-04),FITBVolume(t-04),DVAOpen(t-04),DVAHigh(t-04),DVALow(t-04),DVAClose(t-04),DVAAdj Close(t-04),DVAVolume(t-04),GOOGOpen(t-04),GOOGHigh(t-04),GOOGLow(t-04),GOOGClose(t-04),GOOGAdj Close(t-04),GOOGVolume(t-04),MMCOpen(t-04),MMCHigh(t-04),MMCLow(t-04),MMCClose(t-04),MMCAdj Close(t-04),MMCVolume(t-04),d2close(t-04),d1vol(t-04),d2vol(t-04),M(t-04),T(t-04),W(t-04),Th(t-04),F(t-04),d1close(t-0

In [23]:
# ensure the shaping makes sense the way all the variables are arranged
# reshape input to be 3D [samples(observations), timesteps (n_in), features]
train_X = ps.shape(train_X, n_in=n_in, features=features)
X_validation = ps.shape(X_validation, n_in=n_in, features=features)
test_X = ps.shape(test_X, n_in=n_in, features=features)

In [24]:
print(train_X.shape, X_validation.shape, test_X.shape)

(795, 5, 135) (285, 5, 135) (57, 5, 135)


# Build LSTM Architecture

In [26]:
# for debug on lstm package
import imp
imp.reload(lstm)

<module 'lstm' from 'T:\\Dropbox\\Projects\\LSTM-forecast\\src\\lstm.py'>

In [ ]:
model = lstm.build_model(train_X, loss_function='mean_absolute_percentage_error',
                         timesteps=n_in, 
                         inlayer=int(train_X.shape[-1]*55),
                         hiddenlayers=0, 
                         outlayer=n_out)

# Fit model

In [28]:
# fit network and save to history
history = model.fit(train_X, train_y, 
                    epochs=1500, 
                    batch_size=256, 
                    validation_data=(X_validation, Y_validation), 
                    verbose=2, 
                    shuffle=False)

# plot history
visualize.plot_loss(history)

Train on 795 samples, validate on 285 samples
Epoch 1/1500


ResourceExhaustedError: OOM when allocating tensor with shape[7425,29700]
	 [[Node: training/Adam/mul_6 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Adam/beta_1/read, training/Adam/Variable_1/read)]]
	 [[Node: loss/mul/_93 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1782_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'training/Adam/mul_6', defined at:
  File "C:\Users\hende\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\hende\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\hende\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\hende\Anaconda3\lib\site-packages\traitlets\config\appl

# Visualize Results

In [ ]:
### KEEP THIS WORKING ####
yhat = predicts.predict_sequences_multiple(model, test_X, n_in, n_out)
visualize.plot_results_multiple(yhat, test_y[:,0], n_out, legend=False)
# visualize.plot_single(yhat, test_y, ticker, data_set_category='test')

# Single day predictions

In [ ]:
for i, x in enumerate(yhat):
    print(x, test_y[i,0])

In [ ]:
###### TODO METHOD####
#import predicts
## copy weights
#old_weights=model.get_weights()
#
#new_model = Sequential()
#new_model.add(LSTM(30, input_shape=(train_X.shape[1], train_X.shape[2]),
#          return_sequences=True, activation='tanh'))
#new_model.add(Dropout(0.5))
#new_model.add(LSTM(10, activation='tanh'))
#new_model.add(Dropout(0.5))
#new_model.add(Dense(1))
#new_model.compile(loss='mae', optimizer='adam')
## fit network
#new_model.set_weights(old_weights)
#predicts.onlineForecast(new_model, test_X, test_y)
## # # ____ # # #

# *** Test Reshaping Method

In [ ]:
# test dimension modification
import pandas as pd
df=pd.read_csv('testDim.csv')
features_test=list(df.columns)
df.head()

In [ ]:
n_in=2
n_out=1
tgt_tst='Var4'
df=ps.series_to_supervised(df, 
                           n_in=n_in, 
                           n_out=n_out, 
                           features=features_test)
df.head()

In [ ]:
test_frame=ps.frame_targets(df, 
                          features=features_test, 
                          n_out=n_out,
                          target=tgt_tst)
test_frame.head()

In [ ]:
X = test_frame.values[:, :-n_out]

In [ ]:
[print(x) for x in X]

In [ ]:
d=ps.shape(X, n_in, features_test)

In [ ]:
d[1,-1,:]

* Seems like dim is (observations, n_in, features), which is correct

* tensor should be (t-2)a, (t-2)b, (t-1)a, (t-1)b, etc. where a and b are features to properly reshape, thus current reshaping is working as expected